In [1]:
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time

In [2]:
from selenium import webdriver

driver = webdriver.Chrome()

In [3]:
# Creating python program to search desired keywords one by one and saving links in open list prod_url

# creating open list to save href links of all products 
prod_url = []

# Keywords for search to which we need to scrape the data.
keywords = ['Laptops', 'Phones', 'Headphones', 'Smart Watches', 'Cameras', 'Printers', 'Monitors','Home theater', 'Router']

# website from which we will scrape data
driver.get("https://www.amazon.in/") 

initial_url = driver.current_url

# loop for sending keywords to website and scraping href
for i in keywords:
    # Finding search input and giving input
    search = driver.find_element(By.ID,"twotabsearchtextbox").send_keys(i)
        
    # clicking on search button
    search_btn = driver.find_element(By.XPATH,"//div[@class='nav-search-submit nav-sprite']")
    search_btn.click()
    # giving time to load the webpage completely to avoid errors
    time.sleep(5)
    # finding href of each products mentioned in keywords
    for j in driver.find_elements(By.XPATH,"//h2[@class = 'a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"):
        prod_url.append(j.get_attribute('href'))
    # returning to initial url    
    driver.get(initial_url)
# checking length of prod_url
len(prod_url)

266

In [4]:
# opening each 234 prod_url and scraping links of all reviews href 
# open list to save all reviews href
rev_cli = []

# loop for opening all links stored in prod_url
for j in prod_url:
    # opening each links of prod_url one by one
    driver.get(j)
    
    try:
        # finding links from all webpage by xpath
        for k in driver.find_elements(By.XPATH,"//a[@id = 'acrCustomerReviewLink']"):
            rev_cli.append(k.get_attribute('href'))
    # if all reviews link is not available, it will continue to next page,so avoiding NoSuchElementException 
    except NoSuchElementException as e:
        continue

In [5]:
# Checking length of open list
len(rev_cli)

519

In [6]:
# Now opening each links as stored in list above
# creating open list for storing all reviews href
all_reviews = []

for l in rev_cli:
    # opening each links as stored in cli_rev open list
    driver.get(l)
    time.sleep(3)
    
    try:
        # finding links from all webpage by xpath
        all_rev = driver.find_element(By.XPATH,"//div[@id = 'reviews-medley-footer']/div[2]/a")
        all_reviews.append(all_rev.get_attribute('href'))
        # if all reviews link is not available, it will continue to next page,so avoiding NoSuchElementException 
    except NoSuchElementException as e:
        continue

In [7]:
# Checking length of all reviews link
len(all_reviews)

515

In [8]:
# finally opening all_reviews link and scraping ratings and reviews data from all webpages.
# creating open list to save scraped data
ratings = []
reviews = []

# opening each link to begin the scaping
for m in all_reviews:
    driver.get(m)
    time.sleep(15)
    
    # loop to open link till page 5 and scraping data of reviews and ratings of products
    for n in range(0,5):
        for o in driver.find_elements(By.XPATH,"//div[@class = 'a-section review aok-relative']/div/div/div[2]/a[1]"):
            ratings.append(o.get_attribute('title').replace(' out of 5 stars',''))
            
        for p in driver.find_elements(By.XPATH,"//div[@class = 'a-section review aok-relative']/div/div/div[4]/span/span"):
            reviews.append(p.text)
                        
        try:
            next_page = driver.find_element(By.XPATH,"//ul[@class = 'a-pagination']/li[2]/a")
            driver.get(next_page.get_attribute('href'))
        except NoSuchElementException as e:
            continue

In [9]:
# saving scraped data into dataframe.
search = pd.DataFrame({}) 
search['Ratings'] = ratings[:15000]
search['Full_review'] = reviews[:15000]
search

,Ratings,Full_review
0,4.0,I'm writing this review after using almost a m...
1,5.0,"Touch is good, mouse is good, comes with a st..."
2,1.0,Very bed quality 😴
3,1.0,Its not working properly
4,1.0,Battery Backup Very Poor.\nScreen text get aut...
...,...,...
14995,5.0,Nice
14996,5.0,Good
14997,5.0,Printer is not working
14998,4.0,Good quality and easy to use


In [10]:
# Saving scraped data in csv format
search.to_csv('Ratings_Prediction_Amazon.csv')